In [46]:
import pandas as pd
import folium
import requests
import branca.colormap as cm

# -----------------------------
# 1️⃣ Charger le fichier DVF
# -----------------------------
df = pd.read_csv(
    "ValeursFoncieres.txt",
    sep="|",
    header=0,
    dtype=str,
    low_memory=False
)

cols = [
    "Code departement",
    "Code commune",
    "Section",
    "No plan",
    "No disposition",
    "No voie",
    "Type de voie",
    "Voie",
    "Code postal",
    "Commune",
    "Valeur fonciere",
    "Type local",
    "Surface terrain",
    "Surface reelle bati"
]

df_adresse = df[cols].copy()


In [47]:
# Valeur foncière → float
df_adresse["Valeur fonciere"] = (
    df_adresse["Valeur fonciere"]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

# Sécurité
df_adresse = df_adresse.dropna(
    subset=["Valeur fonciere", "Code departement"]
)

df_adresse["Code departement"] = df_adresse["Code departement"].astype(str)


In [48]:
# Nombre de locaux par mutation
df_adresse["nb_locaux_mutation"] = (
    df_adresse
    .groupby([
        "Code departement",
        "Code commune",
        "Section",
        "No plan",
        "No disposition"
    ])["No disposition"]
    .transform("count")
)

# Valeur foncière répartie par local
df_adresse["valeur_repartie"] = (
    df_adresse["Valeur fonciere"] / df_adresse["nb_locaux_mutation"]
)


In [49]:
tableau_dept = (
    df_adresse
    .groupby("Code departement")
    .agg(
        Valeur_fonciere_moyenne=("valeur_repartie", "mean"),
        nb_biens_par_dept=("Type local", "count"),
        nb_biens_maison=("Type local", lambda x: (x == "Maison").sum()),
        nb_biens_appartement=("Type local", lambda x: (x == "Appartement").sum()),
        nb_biens_dependance=("Type local", lambda x: (x == "Dépendance").sum()),
        nb_biens_commerciaux=("Type local", lambda x: (x == "Local industriel. commercial ou assimilé").sum())
    )
    .reset_index()
)

tableau_dept["Valeur_fonciere_moyenne"] = (
    tableau_dept["Valeur_fonciere_moyenne"].round(0)
)

tableau_dept.to_csv(
    "moyenne_valeur_fonciere_par_departement.csv",
    index=False
)


In [50]:
# GeoJSON départements
url_geojson = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
geojson_data = requests.get(url_geojson).json()

# Carte
carte = folium.Map(
    location=[46.6, 2.5],
    zoom_start=6,
    tiles="cartodbpositron"
)

def format_number(val):
    if isinstance(val, (int, float)):
        return f"{val:,.0f}".replace(",", " ")
    return "N/A"


In [51]:
nb_total = (
    df_adresse
    .groupby("Code departement")
    .size()
    .to_dict()
)

def stats_par_type(type_local):
    df_type = df_adresse[df_adresse["Type local"] == type_local]

    nb = df_type.groupby("Code departement").size().to_dict()

    valeur = (
        df_type
        .groupby("Code departement")["valeur_repartie"]
        .mean()
        .round(0)
        .to_dict()
    )

    return nb, valeur

nb_maison, valeur_maison = stats_par_type("Maison")
nb_appart, valeur_appart = stats_par_type("Appartement")
nb_dep, valeur_dep = stats_par_type("Dépendance")
nb_com, valeur_com = stats_par_type("Local industriel. commercial ou assimilé")


In [52]:
def add_layer(name, nb_dict, valeur_dict):

    valeurs_valides = [v for v in valeur_dict.values() if v and v > 0]
    if not valeurs_valides:
        return

    colormap = cm.LinearColormap(
        colors=["#ffffcc", "#ffeda0", "#feb24c", "#f03b20", "#bd0026"],
        vmin=min(valeurs_valides),
        vmax=max(valeurs_valides),
        caption=f"Valeur foncière moyenne – {name} (€)"
    )
    colormap.add_to(carte)

    layer = folium.FeatureGroup(name=name, show=False)

    for feature in geojson_data["features"]:
        code = feature["properties"]["code"]
        nom = feature["properties"]["nom"]

        nb = nb_dict.get(code, 0)
        total = nb_total.get(code, 1)
        valeur = valeur_dict.get(code)
        pct = (nb / total) * 100 if total else 0

        tooltip_html = (
            f"<b>{nom}</b><br>"
            f"{name} : {format_number(nb)} ({pct:.1f} %)<br>"
            f"Valeur moyenne ({name}) : {format_number(valeur)} €"
        )

        folium.GeoJson(
            feature,
            style_function=lambda f, v=valeur: {
                "fillColor": colormap(v) if v else "#dddddd",
                "color": "black",
                "weight": 0.5,
                "fillOpacity": 0.8
            },
            tooltip=folium.Tooltip(tooltip_html, sticky=True),
            highlight_function=lambda x: {"weight": 3, "color": "blue"}
        ).add_to(layer)

    layer.add_to(carte)


In [53]:
add_layer("Maisons", nb_maison, valeur_maison)
add_layer("Appartements", nb_appart, valeur_appart)
add_layer("Dépendances", nb_dep, valeur_dep)
add_layer("Commerciaux", nb_com, valeur_com)

folium.LayerControl(collapsed=False).add_to(carte)

carte.save("carte_filtrable_biens_par_departement.html")
print("✅ Carte générée : carte_filtrable_biens_par_departement.html")


✅ Carte générée : carte_filtrable_biens_par_departement.html
